In [31]:
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# to make the results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [32]:
class Dataset(torch.utils.data.Dataset):
        def __init__(self, image_files, labels, transforms):
            self.image_files = image_files
            self.labels = labels
            self.transforms = transforms

        def __len__(self):
            return len(self.image_files)

        def __getitem__(self, index):
            return self.transforms(Image.open(self.image_files[index])), self.labels[index]

In [33]:
model_ft = models.densenet121()
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 7)
input_size = 224

In [34]:
PATH="./best_metric_model_skin.pth"
device = torch.device("cpu")

model_ft.load_state_dict(torch.load(PATH, map_location=device))
model_ft.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [39]:
# 예측하고자 하는 사진이 들어있는 경로 = path
def skin_detect(path):
    norm_mean = (0.49139968, 0.48215827, 0.44653124)
    norm_std = (0.24703233, 0.24348505, 0.26158768)
    input_size = 224
    val_x = []
    val_y = []
    prediction = []
    for images in sorted(os.listdir(path), key = len):
        test_image = os.path.join(path, images)
        val_x.append(test_image)
        val_y.append(0)

    val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])
    
    val_ds = Dataset(val_x, val_y, val_transform)

    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0)

    model_ft.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            outputs = model_ft(images)
            pred = outputs.max(1, keepdim=True)[1]
            pred = pred.item()
            prediction.append(pred)
#     if prediction[0] ==0:
#         diagnosis = "뇌종양입니다."
#         img_saved = seg(path)
#         return img_saved, diagnosis
#     elif prediction[0] ==1:
#         diagnosis = "이상 소견 없음"
#         img_saved = None
        return prediction

In [43]:
for i in range(1,11):
    print(skin_detect(f"./images/original/{i}"))

[4]
[0]
[3, 4, 4, 2, 2, 2, 0, 0]
[4]
[4]
[6]
[5]
[4]
[4]
[0]
